In [1]:
# basic
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from tqdm import tqdm
import sklearn

# np/pd
import numpy as np
import pandas as pd

# torch
import torch
import torchtext
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# transformer
from datasets import load_dataset

# CRF
from torchcrf import CRF

In [2]:
from torchtext.vocab import GloVe

global_vectors = GloVe(name='840B', dim=300, cache='../../data/Glove840B300d/')

In [3]:
print(global_vectors.get_vecs_by_tokens(['the','world','will','be','better']).shape)

torch.Size([5, 300])


In [4]:
torch.cuda.is_available()

True

## 探查conll2003数据

定义dataset，dataLoader

In [5]:
# data_udpos = torchtext.datasets.UDPOS(root='./torchtext_datasets_udpos/', split=('train','valid','test'))
dataset_conll2003 = load_dataset("conll2003")

Reusing dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset_conll2003['test'][0]

{'id': '0',
 'tokens': ['SOCCER',
  '-',
  'JAPAN',
  'GET',
  'LUCKY',
  'WIN',
  ',',
  'CHINA',
  'IN',
  'SURPRISE',
  'DEFEAT',
  '.'],
 'pos_tags': [21, 8, 22, 37, 22, 22, 6, 22, 15, 12, 21, 7],
 'chunk_tags': [11, 0, 11, 21, 11, 12, 0, 11, 13, 11, 12, 0],
 'ner_tags': [0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0]}

In [7]:
dataset_conll2003.class_encode_column

<bound method DatasetDict.class_encode_column of DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})>

In [8]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"

In [9]:
ner_tag2id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 
              'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8, START_TAG:9, STOP_TAG:10}

ner_id2tag = {}
for key in ner_tag2id.keys():
    ner_id2tag[ner_tag2id[key]] = key

In [10]:
ner_id2tag

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC',
 9: '<START>',
 10: '<STOP>'}

In [11]:
def ner_id_to_tags(ner_id_seq):
    res1, res2 = [], []
    for ner_id in ner_id_seq:
        res1.append(ner_id2tag.get(ner_id, ''))
        res2.append(ner_id2tag.get(ner_id, '-').split('-')[-1])
    return res1, res2

ner_id_to_tags(dataset_conll2003['train'][0]['ner_tags'])

(['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'],
 ['ORG', 'O', 'MISC', 'O', 'O', 'O', 'MISC', 'O', 'O'])

In [12]:
m = 0
for k in dataset_conll2003.keys():
    for i in dataset_conll2003[k]['tokens']:
        m = max(m, len(i))
print('max length = {}'.format(m))

max length = 124


In [13]:
word_to_ix = {}
for k in dataset_conll2003.keys():
    for tokens in dataset_conll2003[k]['tokens']:
        for token in tokens:
            if token not in word_to_ix:
                word_to_ix[token] = len(word_to_ix)

for token in [START_TAG, STOP_TAG]:
    if token not in word_to_ix:
        word_to_ix[token] = len(word_to_ix)
        
print(len(word_to_ix))

30291


In [14]:
list(word_to_ix.keys())[-3:]

['well-fancied', '<START>', '<STOP>']

In [15]:
t = dataset_conll2003['train'][0]['ner_tags']
t.insert(0, ner_tag2id[START_TAG])
t.append(ner_tag2id[STOP_TAG])
print(t)

[9, 3, 0, 7, 0, 0, 0, 7, 0, 0, 10]


In [16]:
def func_word2ix(word_to_ix, token_list):
    res = list()
    for token in token_list:
        res.append(word_to_ix.get(token, len(word_to_ix)+1))
    return {'token_ids':res}

def fill_start_end_tag(ner_tags_tmp):
    ner_tags_curr = ner_tags_tmp.copy()
    ner_tags_curr.insert(0,  ner_tag2id[START_TAG])
    ner_tags_curr.append(ner_tag2id[STOP_TAG])
    return {'ner_tags_fill': ner_tags_curr}


def func_get_new_cols(word_to_ix, token_list, ner_tags_tmp):
    # get token ids
    res = list()
    res.append(word_to_ix.get(START_TAG))
    for token in token_list:
        res.append(word_to_ix.get(token, len(word_to_ix)+1))
    res.append(word_to_ix.get(STOP_TAG))
    
    # fill the tokens 
    token_list.insert(0, START_TAG)
    token_list.append(STOP_TAG)
    
    # fill the ner tags
    ner_tags_curr = ner_tags_tmp.copy()
    ner_tags_curr.insert(0,  ner_tag2id[START_TAG])
    ner_tags_curr.append(ner_tag2id[STOP_TAG])
    return {'token_ids':res, 'ner_tags_fill': ner_tags_curr, }

In [17]:
word_to_ix.get(START_TAG)

30289

In [18]:
dataset_conll2003_train=dataset_conll2003['train'].map(lambda x: func_get_new_cols(word_to_ix, x['tokens'], x['ner_tags']))
dataset_conll2003_train.set_format(type="torch", columns=['token_ids','ner_tags_fill'])

dataset_conll2003_test=dataset_conll2003['test'].map(lambda x: func_get_new_cols(word_to_ix, x['tokens'], x['ner_tags']))
dataset_conll2003_test.set_format(type="torch", columns=['token_ids','ner_tags_fill'])

dataset_conll2003_val=dataset_conll2003['validation'].map(lambda x: func_get_new_cols(word_to_ix, x['tokens'], x['ner_tags']))
dataset_conll2003_val.set_format(type="torch", columns=['token_ids','ner_tags_fill'])

Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-98d7a3e3f12bd0c0.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-ac8336dbe7d3dd9f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-7798d4403bff3e48.arrow


In [19]:
dataset_conll2003_test['ner_tags_fill'][1]

tensor([ 9,  1,  2, 10])

In [20]:
dataset_conll2003_train['tokens'][0], dataset_conll2003_train['token_ids'][0],  dataset_conll2003_train['ner_tags_fill'][0]

(['<START>',
  'EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.',
  '<STOP>'],
 tensor([30289,     0,     1,     2,     3,     4,     5,     6,     7,     8,
         30290]),
 tensor([ 9,  3,  0,  7,  0,  0,  0,  7,  0,  0, 10]))

In [21]:
dataset_conll2003_train

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'token_ids', 'ner_tags_fill'],
    num_rows: 14041
})

In [22]:
class MyDataset(Dataset):

    def __init__(self, data):
        self.data = data
        self.token_ids = self.data['token_ids'] # 在这变成torch.tensor，但长度不同
        self.ner_tags_fill = self.data['ner_tags_fill']
        self.tokens = self.data['tokens']
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
#         curr = dict()
#         curr['token_ids'] = self.token_ids
#         curr['ner_tags'] = self.ner_tags
        return self.token_ids[index], self.ner_tags_fill[index], self.tokens[index]


def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    x, y, z = zip(*batch)
    x_lens = [len(x_i) for x_i in x]
    y_lens = [len(y_i) for y_i in y]
    x_pad = torch.nn.utils.rnn.pad_sequence(x, batch_first=True)
    y_pad = torch.nn.utils.rnn.pad_sequence(y, batch_first=True)
    return x_pad, torch.tensor(x_lens), y_pad, torch.tensor(y_lens), z
    
dataset_conll2003_train_loader = DataLoader(
    MyDataset(dataset_conll2003_train),
    batch_size=32,
    shuffle=True, 
    collate_fn=lambda x: collate_fn_padd(x))


dataset_conll2003_test_loader = DataLoader(
    MyDataset(dataset_conll2003_test),
    batch_size=32,
    shuffle=True, 
    collate_fn=lambda x: collate_fn_padd(x))

In [96]:
dataset_conll2003_val_loader = DataLoader(
    MyDataset(dataset_conll2003_val),
    batch_size=32,
    shuffle=True, 
    collate_fn=lambda x: collate_fn_padd(x))

In [23]:
t = next(iter(dataset_conll2003_train_loader))
pos = 0
print(t[0][pos], sum(t[0][pos]>0), t[1][pos], t[2][pos], t[3][pos], t[4][pos], len(t[4][pos]))

tensor([30289,  4936,    70,  4935,  2058, 30290,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0]) tensor(6) tensor(6) tensor([ 9,  5,  0,  5,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]) tensor(6) ['<START>', 'PARAMARIBO', ',', 'Surinam', '1996-08-21', '<STOP>'] 6


In [24]:
print(dataset_conll2003_train_loader.dataset.__len__(), dataset_conll2003_test_loader.dataset.__len__())

14041 3453


## token-emb

In [25]:
print(global_vectors.get_vecs_by_tokens(START_TAG)[0:10],global_vectors.get_vecs_by_tokens(STOP_TAG)[0:10] )

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [55]:
glove_weight = global_vectors.get_vecs_by_tokens(list(word_to_ix.keys()))
glove_weight[-2], glove_weight[-1] = torch.randn(300), torch.randn(300)
embedding_glove = nn.Embedding.from_pretrained(glove_weight)

In [59]:
embedding_glove(torch.tensor(glove_weight.shape[0]-1))

tensor([ 9.4590e-01,  6.4280e-01,  1.2850e+00, -1.9312e+00, -7.7402e-01,
         1.1995e+00, -1.1461e+00, -7.8981e-01,  2.0956e-02,  2.0498e+00,
        -1.9247e-01,  1.1378e+00,  3.1604e-01,  7.2349e-01,  1.0631e+00,
         4.6080e-01, -2.2031e+00, -1.6780e-01, -5.1149e-01,  7.7356e-01,
         2.8797e-01, -2.8857e-01, -1.8801e+00, -1.0674e+00,  2.7922e-01,
        -4.4782e-01,  2.5344e-01,  3.0045e-02,  3.5837e-01,  2.1979e-01,
        -1.1757e+00, -9.0621e-02, -8.6704e-01,  3.3297e-02,  6.8099e-01,
         1.1265e-01, -1.2839e+00, -4.6444e-01,  1.3207e+00,  6.8688e-01,
        -8.0311e-01, -5.2655e-01, -9.6999e-01,  7.4705e-01,  8.9972e-01,
         7.6337e-02, -1.3263e+00, -5.2868e-01, -1.2421e+00,  2.5707e+00,
        -4.9449e-01, -7.2044e-02,  1.3960e+00, -5.3946e-01, -3.2282e-01,
         7.2201e-01,  1.5926e+00, -3.4144e-01, -1.3687e+00,  4.3110e-01,
         1.6970e+00, -3.4245e-01,  8.4286e-01, -1.1194e+00, -9.9454e-01,
        -1.2910e+00, -2.3693e+00, -9.6196e-01,  1.8

In [60]:
(1-glove_weight[0]==embedding_glove(torch.LongTensor([0]))).sum()

tensor(0)

## 模型定义

In [72]:
class GLove_BiLSTM_CRF(nn.Module):

    def __init__(self, config=None, embedding_weight = None):
        super(GLove_BiLSTM_CRF, self).__init__()
        self.config = config

        # BiLSTM-model 给 emission 层定义参数
        self.embedding_dim = self.config.get('embedding_dim', 300)
        self.hidden_dim = self.config.get('hidden_dim', 200)
        self.vocab_size = self.config.get('vocab_size', 30289)

        if embedding_weight is not None:
            self.word_embeds = nn.Embedding.from_pretrained(embedding_weight)
        else:
            self.word_embeds = nn.Embedding(self.vocab_size, self.embedding_dim)            
            
        self.target_size = self.config.get('num_tags', 11)
        self.num_layers = self.config.get('num_layers',1)
        self.batch_size = self.config.get('batch_size',16)
        self.bidirectional = True

        # lstm
        self.lstm = nn.LSTM(input_size=self.embedding_dim, hidden_size=self.hidden_dim//2,
                            num_layers=self.num_layers, bidirectional=self.bidirectional)
        self.hidden2tag1 = nn.Linear(self.hidden_dim, self.target_size*3)
        self.hidden2tag2 = nn.Linear(self.target_size*3, self.target_size)
        self.dropout010 = nn.Dropout(0.1)
        self.dropout020 = nn.Dropout(0.2)
#         self.hidden_init = self.init_hidden()

        # CRF-model
        self.crf = CRF(self.config.get('num_tags', 9), batch_first=True)
        self.crf.transitions.data[ner_tag2id[START_TAG], :] = -10000
        self.crf.transitions.data[:, ner_tag2id[STOP_TAG]] = -10000

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers*2 if self.bidirectional else self.num_layers,
                             self.batch_size, self.hidden_dim//2)
        cell = torch.zeros(self.num_layers*2 if self.bidirectional else self.num_layers,
                             self.batch_size, self.hidden_dim//2)
        return hidden, cell
    
    def cal_mask(self, sent_len):
        sent_len = sent_len.cuda()
        batch_size = torch.tensor(len(sent_len))
        batch_size = batch_size.cuda()
        batch_seq_len_max = max(sent_len)
        mask = torch.zeros((batch_size.item(), batch_seq_len_max))
        mask = mask.cuda()
        for mask_i in range(mask.shape[0]):
            mask[mask_i][0:sent_len[mask_i]] = 1
#         assert sum(mask.sum(axis=1) == sent_len) // batch_size == 1
        mask = mask > 0
        return mask
    
    def _get_features(self, sent, sent_len):
#         mask = self.cal_mask(sent_len)
            
        embeds = self.word_embeds(sent)
        embed_packed = pack_padded_sequence(embeds, lengths=sent_len.to('cpu'),
                                            batch_first=True,
                                            enforce_sorted=False)
        lstm_out, (hidden, cell) = self.lstm(embed_packed) #, self.hidden_init)
        lstm_out, lens = pad_packed_sequence(lstm_out, batch_first=True)
        tag_score = self.hidden2tag1(lstm_out)
        tag_score = self.dropout010(tag_score)
        tag_score = self.hidden2tag2(tag_score)
        return tag_score
    
    def neg_log_likelihood(self, sent, label, sent_len):
        assert sent.shape[0:2]==label.shape
        mask = self.cal_mask(sent_len)
        mask = mask.cuda()
        feats = self._get_features(sent, sent_len)
        return self.crf(feats, label, mask)

    def forward(self, sent, sent_len):
        """

        :param sent: 输入的已转换为token_id的句子，(batch_len * sent_len * token_emb_len)
        :param sent_len: tensor(list(int))
        :return:
        """
        feats = self._get_features(sent, sent_len)
        return torch.tensor(self.crf.decode(feats))
#         batch_size = len(sent_len)
#         batch_seq_len_max = max(sent_len)
#         mask = torch.zeros((batch_size, batch_seq_len_max))
#         for mask_i in range(mask.shape[0]):
#             mask[mask_i][0:sent_len[mask_i]] = 1
#         assert sum(mask.sum(axis=1) == sent_len) // batch_size == 1
#         mask = self.cal_mask(sent_len)
            
#         embeds = self.word_embeds(sent)
#         embed_packed = pack_padded_sequence(embeds, lengths=sent_len.to('cpu'),
#                                             batch_first=True,
#                                             enforce_sorted=False)
#         lstm_out, (hidden, cell) = self.lstm(embed_packed) #, self.hidden_init)
#         lstm_out, lens = pad_packed_sequence(lstm_out, batch_first=True)
#         tag_score = self.hidden2tag1(lstm_out)
#         tag_score = self.dropout010(tag_score)
#         tag_score = self.hidden2tag2(tag_score)
#         tag_score = self.crf(tag_score)
# #         tag_score = nn.functional.softmax(tag_score, dim=-1)
#         return tag_score

In [62]:
Config = {
    'num_tags':len(ner_tag2id),
    'num_layers':1,
    'embedding_dim':300,
    'vocab_size':glove_weight.shape[0],
    'hidden_dim':100,
    'batch_size':32,
    'START_TAG':"<START>",
    'STOP_TAG': "<STOP>"
}

Config

{'num_tags': 11,
 'num_layers': 1,
 'embedding_dim': 300,
 'vocab_size': 30291,
 'hidden_dim': 100,
 'batch_size': 32,
 'START_TAG': '<START>',
 'STOP_TAG': '<STOP>'}

In [63]:
model_name = 'V3-Emb=Glove300-bilstmCRFLayer=1Hidden=200Dropout0.1Batch=32Learn=1e-1'
model_lstm_crf = GLove_BiLSTM_CRF(config=Config, embedding_weight=glove_weight)

model_lstm_crf = model_lstm_crf.cuda()

model_lstm_crf.parameters

<bound method Module.parameters of GLove_BiLSTM_CRF(
  (word_embeds): Embedding(30291, 300)
  (lstm): LSTM(300, 50, bidirectional=True)
  (hidden2tag1): Linear(in_features=100, out_features=33, bias=True)
  (hidden2tag2): Linear(in_features=33, out_features=11, bias=True)
  (dropout010): Dropout(p=0.1, inplace=False)
  (dropout020): Dropout(p=0.2, inplace=False)
  (crf): CRF(num_tags=11)
)>

In [64]:
# 手动计算验证权重
import collections
ner_tags_all = torch.cat(dataset_conll2003_train['ner_tags_fill'])
t=collections.Counter(ner_tags_all.numpy())
res = []
for k in t:
    res.append((k, len(ner_tags_all)/9/t[k]))
print(res)

class_weights=sklearn.utils.class_weight.compute_class_weight(
    class_weight='balanced',classes=np.unique(ner_tags_all),y=ner_tags_all.numpy())
class_weights=torch.tensor(class_weights,dtype=torch.float)
class_weights

[(9, 1.8335430366624725), (3, 4.072896341999332), (0, 0.15181673199222645), (7, 7.488300691616573), (10, 1.8335430366624725), (1, 3.900723905723906), (2, 5.685684138201806), (5, 3.605711173358232), (4, 6.950533957283417), (8, 22.28985088985089), (6, 22.251320464803612)]


tensor([ 0.1242,  3.1915,  4.6519,  3.3324,  5.6868,  2.9501, 18.2056,  6.1268,
        18.2372,  1.5002,  1.5002])

In [65]:
optimizer = torch.optim.Adam(model_lstm_crf.parameters(), lr=1e-1)
loss_fn = nn.CrossEntropyLoss(reduction='mean', weight=class_weights.cuda()) 
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

## 测试model

In [66]:
# # 测试model
# tmp = next(iter(dataset_conll2003_train_loader))
# # print(tmp)
# sent = tmp[0]
# print(sent.shape)
# sent_len = tmp[1]
# y = tmp[2]

# print(sent.shape, y.shape)

# sent = sent.cuda()
# sent_len = sent_len.cuda()

# model_lstm_crf = model_lstm_crf.cuda()

# with torch.no_grad():
#     res1 = model_lstm_crf(sent.cuda(), sent_len.cuda())
#     loss = model_lstm_crf.neg_log_likelihood(sent=sent, label=y, sent_len=sent_len)

## 定义训练函数

In [67]:
def train_eval_single_epoch(model, data_iter=None, optimizer=None, loss_fn=None, is_train=False):
    if is_train:
        model.train()
    else:
        model.eval()   
    correct_curr, correct_sum, loss_sum, loss_curr = 0, 0, 0, 0
    loss_list, accuracy_list = [], []
    print('Total (training) batch = {}'.format(len(data_iter)))
    batch_i = 0
    data_iter_len = data_iter.dataset.__len__() # total sample num.
    batch_num = len(data_iter)
    
    batch_loss_list = list()
    logits_list = list()
    y_list, y_len_list = [], []
    for batch_data in data_iter:
        torch.cuda.empty_cache()
        batch_i += 1
        if is_train:
            optimizer.zero_grad()
            model.zero_grad()
        x, x_len, y, y_len, _ = batch_data
        x = x.cuda()
        x_len = x_len.cuda()
        y = y.cuda()
        # model predict 
        logits = model(x, x_len)
        assert logits.shape[0:2]==x.shape[0:2]
        # compute loss 
#         batch_loss = 0
        batch_loss = - model.neg_log_likelihood(sent=x, label=y, sent_len=x_len)
#         for i in range(logits.size(0)): # num of samples in one batch
#             loss = loss_fn(logits[i], y[i])
#             batch_loss += loss 
        
#         batch_loss /= logits.size(0)
        if is_train:
            batch_loss.backward()
            optimizer.step()
        
        # 记录
        batch_loss_list.append(batch_loss.item())
        logits_list.append(logits)
        y_list.append(y)
        y_len_list.append(y_len)
#         if batch_i%100==0:
#             print(batch_loss.item())
        
        x = x.cpu()
        x_len = x_len.cpu()
        y = y.cpu()
        
#     print(sum(batch_loss_list)/batch_num)
    return batch_loss_list, logits_list, y_list, y_len_list

In [68]:
def func_eval(model, data_iter=None, loss_fn=None):
    model = model.cuda()
    model.eval()
    correct_curr, correct_sum, loss_sum, loss_curr = 0, 0, 0, 0
    loss_list, accuracy_list = [], []
    print('Total (training) batch = {}'.format(len(data_iter)))
    batch_i = 0
    data_iter_len = data_iter.dataset.__len__() # total sample num.
    batch_num = len(data_iter)
    
    batch_loss_list = list()
    logits_list = list()
    y_list, y_len_list = [], []
    for batch_data in data_iter:
        batch_i += 1
        x, x_len, y, y_len,_ = batch_data
        x = x.cuda()
        x_len = x_len.cuda()
        y = y.cuda()
        # model predict 
        logits = model(x, x_len)
        assert logits.shape[0:2]==x.shape[0:2]
        # compute loss 
        batch_loss = - model.neg_log_likelihood(sent=x, label=y, sent_len=x_len)
#         batch_loss = 0
#         for i in range(logits.size(0)): # num of samples in one batch
#             loss = loss_fn(logits[i], y[i])
#             batch_loss -= loss 
        
#         batch_loss /= logits.size(0)

        # 记录
        batch_loss_list.append(batch_loss.item())
        logits_list.append(logits)
        y_list.append(y)
        y_len_list.append(y_len)
#         if batch_i%100==0:
#             print(batch_loss.item())
        
        x = x.cpu()
        x_len = x_len.cpu()
        y = y.cpu()
        
#     print(sum(batch_loss_list)/batch_num)
    return batch_loss_list, logits_list, y_list, y_len_list

## 训练开始

In [69]:
from MyFunctions3 import MyTools

In [70]:
f1_test_curr, f1_test_max = 0, 0

for epoch in tqdm(range(50)):
    print('='*50)
    print('Epoch = {}'.format(epoch))
    print('='*50)
    batch_loss_list, logits_list, y_list, y_len_list = train_eval_single_epoch(model_lstm_crf, 
                                                                               dataset_conll2003_train_loader, 
                                                                               optimizer=optimizer,
                                                                               loss_fn=loss_fn,
                                                                               is_train=True)
    scheduler.step() # 加上后好一些
    print('training loss = {:.8f}'.format(sum(batch_loss_list)/len(batch_loss_list)))
    test_batch_loss_list, test_logits_list, test_y_list, test_y_len_list = func_eval(model_lstm_crf, 
                                                                               dataset_conll2003_test_loader, 
                                                                               loss_fn=loss_fn)
    print('testing loss = {:.8f}'.format(sum(test_batch_loss_list)/len(test_batch_loss_list)))
    
#     评估
    train_dict = MyTools.func_cal_accu_recall(logits_list=logits_list, y_list=y_list, y_len_list=y_len_list, 
                                              is_logits_tag=True)
    test_dict = MyTools.func_cal_accu_recall(logits_list=test_logits_list, y_list=test_y_list, 
                                             y_len_list=test_y_len_list, is_logits_tag=True)
    
    print('Train metrics')
    train_metric = MyTools.func_cal_metrics(train_dict)
    print('Test metrics')
    test_metric = MyTools.func_cal_metrics(test_dict)
    
    # save model if current f1 rate is better than previous ones
    recall_test_curr, precision_test_curr, accuracy_test_curr, f1_test_curr = test_metric
    if f1_test_curr > f1_test_max:
        torch.save(model_lstm_crf.state_dict(), './models/model_v3_'+model_name+'epoch='+str(epoch)+
                   'F1='+str(round(f1_test_curr,4))+
                   'accu='+str(round(accuracy_test_curr,4))+
                   'recall='+str(round(recall_test_curr,4))+
                   'precision='+str(round(precision_test_curr,4)))
    f1_test_max = max(f1_test_curr, f1_test_max)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch = 0
Total (training) batch = 439
training loss = 37084.86910149
Total (training) batch = 108
testing loss = 188.79130554


/home/web_server/antispam/project/kml_public/liuguizhou/my_train/POS/conll2003/MyFunctions3.py:25: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  assert sum(mask.sum(axis=1) == y_len_list[logit_i]) // batch_len == 1
  2%|▏         | 1/50 [01:03<51:31, 63.10s/it]

Train metrics
recall = 51.67%, precision = 75.46%, accuracy = 89.11%, f1 = 61.34%
Test metrics
recall = 63.24%, precision = 87.93%, accuracy = 92.06%, f1 = 73.57%
Epoch = 1
Total (training) batch = 439
training loss = 242.62431958
Total (training) batch = 108
testing loss = 172.03254982


  4%|▍         | 2/50 [02:05<50:00, 62.51s/it]

...

Train metrics
recall = 4.90%, precision = 7.95%, accuracy = 74.62%, f1 = 6.06%
Test metrics
recall = 7.27%, precision = 33.91%, accuracy = 81.32%, f1 = 11.98%
Epoch = 48
Total (training) batch = 439
training loss = 2786.42918921
Total (training) batch = 108
testing loss = 292.75405093


 98%|█████████▊| 49/50 [51:04<01:01, 61.65s/it]

Train metrics
recall = 4.51%, precision = 11.12%, accuracy = 78.01%, f1 = 6.42%
Test metrics
recall = 9.95%, precision = 30.38%, accuracy = 80.29%, f1 = 14.99%
Epoch = 49
Total (training) batch = 439
training loss = 2992.71072395
Total (training) batch = 108
testing loss = 268.07975260


100%|██████████| 50/50 [52:06<00:00, 62.54s/it]

Train metrics
recall = 3.98%, precision = 9.50%, accuracy = 77.61%, f1 = 5.61%
Test metrics
recall = 2.55%, precision = 34.73%, accuracy = 82.14%, f1 = 4.75%


## 训练总结

不太清楚为何到后来，训练集开始出现recall和precision的波动；最后下降到10%以下（就很离谱？！）

## 模型加载及结果探查

我们使用第8个epoch模型，
- test： recall = 82.04%, precision = 92.75%, accuracy = 95.74%, f1 = 87.07%
- eval： recall = 83.20%, precision = 96.40%, accuracy = 96.67%, f1 = 89.32%



In [73]:
model_load = GLove_BiLSTM_CRF(config=Config)
model_load.load_state_dict(torch.load('./models/model_v3_V3-Emb=Glove300-bilstmCRFLayer=1Hidden=200Dropout0.1Batch=32Learn=1e-1epoch=8F1=0.8707accu=0.9574recall=0.8204precision=0.9275'))

<All keys matched successfully>

In [74]:
model_load

GLove_BiLSTM_CRF(
  (word_embeds): Embedding(30291, 300)
  (lstm): LSTM(300, 50, bidirectional=True)
  (hidden2tag1): Linear(in_features=100, out_features=33, bias=True)
  (hidden2tag2): Linear(in_features=33, out_features=11, bias=True)
  (dropout010): Dropout(p=0.1, inplace=False)
  (dropout020): Dropout(p=0.2, inplace=False)
  (crf): CRF(num_tags=11)
)

In [94]:
test_batch_loss_list_load, test_logits_list_load, test_y_list_load, test_y_len_list_load = func_eval(model_load, 
                                                                           dataset_conll2003_test_loader, 
                                                                           loss_fn=loss_fn)
print('testing loss = {:.8f}'.format(sum(test_batch_loss_list_load)/len(test_batch_loss_list_load)))

#     评估
test_dict_load = MyTools.func_cal_accu_recall(logits_list=test_logits_list_load, y_list=test_y_list_load, 
                                         y_len_list=test_y_len_list_load, is_logits_tag=True)
test_metric_load = MyTools.func_cal_metrics(test_dict_load)

Total (training) batch = 108
testing loss = 79.28989290
recall = 82.04%, precision = 92.75%, accuracy = 95.74%, f1 = 87.07%


In [97]:
eval_batch_loss_list_load, eval_logits_list_load, eval_y_list_load, eval_y_len_list_load = func_eval(model_load, 
                                                                           dataset_conll2003_val_loader, 
                                                                           loss_fn=loss_fn)
print('evaling loss = {:.8f}'.format(sum(eval_batch_loss_list_load)/len(eval_batch_loss_list_load)))

#     评估
eval_dict_load = MyTools.func_cal_accu_recall(logits_list=eval_logits_list_load, y_list=eval_y_list_load, 
                                         y_len_list=eval_y_len_list_load, is_logits_tag=True)
eval_metric_load = MyTools.func_cal_metrics(eval_dict_load)

Total (training) batch = 102
evaling loss = 66.94167350


/home/web_server/antispam/project/kml_public/liuguizhou/my_train/POS/conll2003/MyFunctions3.py:25: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  assert sum(mask.sum(axis=1) == y_len_list[logit_i]) // batch_len == 1


recall = 83.20%, precision = 96.40%, accuracy = 96.67%, f1 = 89.32%


In [93]:
test_dict_load

{'tp': 6655.0,
 'tn': 37803.0,
 'fp': 520.0,
 'fn': 500.0,
 'others': 957.0,
 'n_total': 46435.0}

In [90]:
t=next(iter(dataset_conll2003_test_loader))
# print(t)
model_load.cuda()
model_load.eval()
y = t[2]
res_arg = model_load(t[0].cuda(), t[1].cuda())

(tensor([[30289,  1831, 30290,  ...,     0,     0,     0],
        [30289,  1910,   389,  ...,     0,     0,     0],
        [30289,  2342,   318,  ...,     0,     0,     0],
        ...,
        [30289,   269, 11672,  ...,     0,     0,     0],
        [30289,  1428,  3760,  ...,     0,     0,     0],
        [30289,  2260,  5889,  ...,     0,     0,     0]]), tensor([ 3, 11, 10, 25,  5, 52,  9,  9,  4,  4,  9, 28,  7,  9, 33, 10,  9, 16,
        37, 43, 36, 12,  4, 27,  9, 23, 10,  8, 15, 43,  6, 22]), tensor([[ 9,  5, 10,  ...,  0,  0,  0],
        [ 9,  0,  0,  ...,  0,  0,  0],
        [ 9,  3,  0,  ...,  0,  0,  0],
        ...,
        [ 9,  0,  0,  ...,  0,  0,  0],
        [ 9,  3,  0,  ...,  0,  0,  0],
        [ 9,  5,  6,  ...,  0,  0,  0]]), tensor([ 3, 11, 10, 25,  5, 52,  9,  9,  4,  4,  9, 28,  7,  9, 33, 10,  9, 16,
        37, 43, 36, 12,  4, 27,  9, 23, 10,  8, 15, 43,  6, 22]), (['<START>', 'Australia', '<STOP>'], ['<START>', 'Scorers', ':', 'Shkvyrin', 'Igor', '78'

### 案例探查总结

一些误判如下：
- 一些容易修正的误判（加规则或者加CRF）
's, y: O, predict: I-ORG

- 地点误判为机构
United, y: B-LOC, predict: I-ORG Arab, y: I-LOC, predict: I-LOC Emirates, y: I-LOC, predict: I-ORG

In [91]:
# true value
y_ner_list = []
for sent in y:
    tmp = []
    for token in sent:
        tmp.append(ner_id2tag[token.item()])
    y_ner_list.append(tmp)

# predict
res_ner_list = []
for sent in res_arg:
    tmp = []
    for token in sent:
        tmp.append(ner_id2tag[token.item()])
    res_ner_list.append(tmp)

for i, sent in enumerate(t[4]):
    print('='*50)
    for j, token in enumerate(sent):
        print('{}, y: {}, predict: {}'.format(token, y_ner_list[i][j], res_ner_list[i][j]))

<START>, y: <START>, predict: <START>
Australia, y: B-LOC, predict: B-LOC
<STOP>, y: <STOP>, predict: <STOP>
<START>, y: <START>, predict: <START>
Scorers, y: O, predict: O
:, y: O, predict: O
Shkvyrin, y: B-PER, predict: B-PER
Igor, y: I-PER, predict: I-PER
78, y: O, predict: O
,, y: O, predict: O
Shatskikh, y: B-PER, predict: B-PER
Oleg, y: I-PER, predict: I-PER
90, y: O, predict: O
<STOP>, y: <STOP>, predict: <STOP>
<START>, y: <START>, predict: <START>
Everton, y: B-ORG, predict: B-ORG
16, y: O, predict: O
6, y: O, predict: O
6, y: O, predict: O
4, y: O, predict: O
25, y: O, predict: O
20, y: O, predict: O
24, y: O, predict: O
<STOP>, y: <STOP>, predict: <STOP>
<START>, y: <START>, predict: <START>
In, y: O, predict: O
another, y: O, predict: O
attack, y: O, predict: O
,, y: O, predict: O
also, y: O, predict: O
on, y: O, predict: O
the, y: O, predict: O
province, y: O, predict: O
's, y: O, predict: O
south, y: O, predict: O
coast, y: O, predict: O
on, y: O, predict: O
Thursday, y: 